# Parameters

> Define a spctial class with is easy you use for config/parametrs 

In [ ]:
#| default_exp parameters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json

In [ ]:
#| export
class Parameters():
    '''A splecial class whos atributes can be referenced as attributs or as dictionaty keys'''

    def __init__(self, **kargs):
        self.add_attr(**kargs)

    def __call__(self, param, value=None):
        if value is not None:
            setattr(self, param, value)
        return self.__getattribute__(param)

    def __repr__(self):
        str_out = f'{self.__class__}'
        for key, value in self.__dict__.items():
            str_out += f'\n   {key} : '+'\n   '.join([i for i in repr(value).split('\n')]) if isinstance(value,Parameters) else f'\n   {key} : {repr(value)}'
        return str_out

    def __str__(self):
        str_out = 'Parameters:'
        for key,value in self.__dict__.items():
            str_out += f'\n   {key} : '+'\n   '.join([i for i in str(value).split('\n')]) if isinstance(value,Parameters) else f'\n   {key} : {value}'
        return str_out

    def add_attr(self, **kargs):
        '''Add attributes to the Parameters class, this will be done recursivly'''
        for key, value in kargs.items():
            if isinstance(value, dict):
                setattr(self, key, Parameters(**value))
            else:
                setattr(self, key, value)

    def to_dict(self):
        '''Convert the parameters to dictionary recorsively'''
        return {key:value if not isinstance(value,Parameters) else value.to_dict() for key,value in self.__dict__.items()}

    def __len__(self) -> int:
        return len(self.__dict__ )

    def __getitem__(self, key):
        return self.__getattribute__(key)

    def __setitem__(self, key, value):
        self.add_attr(**{key: value})

    def __delitem__(self, key):
        delattr(self, key)

    def from_json(self, json_file_name:str):
        '''Read json file and add to parameters'''
        with open(json_file_name) as json_data_file:
            data = json.load(json_data_file)
        self.add_attr(**data)
        return self


In [ ]:
show_doc(Parameters)

---

[source](https://github.com/yuval6957/reinautils/blob/main/reinautils/parameters.py#L10){target="_blank" style="float:right; font-size:smaller"}

### Parameters

>      Parameters (**kargs)

A splecial class whos atributes can be referenced as attributs or as dictionaty keys

In [ ]:
show_doc(Parameters.add_attr,title_level=4)

---

[source](https://github.com/yuval6957/reinautils/blob/main/reinautils/parameters.py#L33){target="_blank" style="float:right; font-size:smaller"}

#### Parameters.add_attr

>      Parameters.add_attr (**kargs)

Add attributes to the Parameters class, this will be done recursivly

In [ ]:
show_doc(Parameters.to_dict,title_level=4)

---

[source](https://github.com/yuval6957/reinautils/blob/main/reinautils/parameters.py#L41){target="_blank" style="float:right; font-size:smaller"}

#### Parameters.to_dict

>      Parameters.to_dict ()

Convert the parameters to dictionary recorsively

In [ ]:
show_doc(Parameters.from_json,title_level=4)

---

[source](https://github.com/yuval6957/reinautils/blob/main/reinautils/parameters.py#L57){target="_blank" style="float:right; font-size:smaller"}

#### Parameters.from_json

>      Parameters.from_json (json_file_name:str)

Read json file and add to parameters

## Use cases

We can instantiate a Parameter class and immediately add attributes 

In [ ]:
params=Parameters(first=1,second='A')
assert (params.first==1) & (params.second=='A')

Attributes can be added later

In [ ]:
params.added = 'I am new'
assert params['added'] == 'I am new'

And they can also be added recursively 

In [ ]:
params.add_attr(file_name='test.ini', paths = {'path1':'hello_world', 'path2':'http2'})
assert params.file_name == 'test.ini'
assert params.paths.path2 == 'http2'

You can see we can referance the attribute directly as in dict

In [ ]:
assert params.paths.path1 == params['paths']['path1']
params['stam'] = 'no'
assert params.stam == 'no'
assert params['paths'].path2 == 'http2'

And can be deleted

In [ ]:
del params['stam']
assert not hasattr(params,'stam')

The Parameters class can be printed and can be converted recursively to dict

In [ ]:
print(params)

Parameters:
   first : 1
   second : A
   added : I am new
   file_name : test.ini
   paths : Parameters:
      path1 : hello_world
      path2 : http2


In [ ]:
print(params.to_dict())

{'first': 1, 'second': 'A', 'added': 'I am new', 'file_name': 'test.ini', 'paths': {'path1': 'hello_world', 'path2': 'http2'}}


The parameters can also be populated using a json file

In [ ]:
params2=Parameters().from_json('config_demo.json')

In [ ]:
print(params2)

Parameters:
   path : Parameters:
      data : /workspace/hd/
      tmp : /workspace/hd/tmp/
      features : /workspace/nvme/features/
      train : /workspace/nvme/train/
      models : /workspace/hd/models/
      output : /workspace/hd/outputs/
      test : /workspace/nvme/test/
   platform : myserver


In [ ]:
assert params2.platform == 'myserver'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()